In [7]:
## Nice link to se K-mean: https://ivanludvig.github.io/kmeans/

# Be sure to pip install: pandas, NumPy, scikit-learn, Seaborn and Matplotlib.
# Dependencies
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt

# Load the train and test datasets to create two DataFrames
train_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv"
train = pd.read_csv(train_url)
test_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv"
test = pd.read_csv(test_url)


In [8]:

## =======================================================================================
# # print("***** Train_Set *****")
# # print(train.head())
# # print("\n")
# # print("***** Test_Set *****")
# # print(test.head())

# ## You can get some initial statistics of both the train and test DataFrames using pandas'
# ## describe()method.
# # print("***** Train_Set *****")
# # print(train.describe())

# # print("***** Values in train_set *****")
# # print(train.columns.values)

# ## What do we do when we have missing data (NaN)?
# ## For the train set
# train.isna().head()
# ## For the test set
# test.isna().head()
# ## Let's get the total number of missing values in both datasets.
# print("*****In the train set*****")
# print(train.isna().sum())
# print("\n")
# print("*****In the test set*****")
# print(test.isna().sum())

# ## There are a couple of ways to handle missing values:
# ## • Remove rows with missing values
# ## • Add value to missing values (best option)

# ## Now, there are several ways you can perform the imputation:
# ## • A constant value that has meaning within the domain, such as 0, distinct from all other values.
# ## • A value from another randomly selected record.
# ## • A mean, median or mode value for the column.
# ## • A value estimated by another machine learning model.


# ## Mean Imputation (adding missing values)
# ## Fill missing values with mean column values in the train set
# # train.fillna(train.mean(), inplace=True) # Fungerar inte för tillfället..?
# ## Fill missing values with mean column values in the test set
# # test.fillna(test.mean(), inplace=True)
# ## OBS! Values that are non-numeric wont get changed in the code above
# ""
# print("*****Sum is NaN train*****")
# print(train.isna().sum())
# print("*****Sum is NaN test*****")
# print(test.isna().sum())


# print("Survival count with respect to Pclass:")
# ## Survival count with respect to Pclass:
# print(
#     train[["Pclass", "Survived"]]
#     .groupby(["Pclass"], as_index=False)
#     .mean()
#     .sort_values(by="Survived", ascending=False)
# )

# print("Survival count with respect to Sex:")
# print(
#     train[["Sex", "Survived"]]
#     .groupby(["Sex"], as_index=False)
#     .mean()
#     .sort_values(by="Survived", ascending=False)
# )

# print("Survival count with respect to SibSps:")
# # Survival count with respect to SibSp:
# print(
#     train[["SibSp", "Survived"]]
#     .groupby(["SibSp"], as_index=False)
#     .mean()
#     .sort_values(by="Survived", ascending=False)
# )

# g = sns.FacetGrid(train, col="Survived")
# g.map(plt.hist, "Age", bins=20)
# plt.show()

## =======================================================================================
print("\n")
train.info()
print("\n")
print("*****In the train set, nr of NaN*****")
print(train.isna().sum())
print("\n")
print("*****In the test set, nr of NaN*****")
print(test.isna().sum())
print("\n")

train['Age'].fillna(train['Age'].mean(), inplace=True)
test['Age'].fillna(test['Age'].mean(), inplace=True)

print("\n")
print("*****New*****")
print("*****In the train set, nr of NaN*****")
print(train.isna().sum())
print("\n")
print("*****In the test set, nr of NaN*****")
print(test.isna().sum())
print("\n")




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


*****In the train set, nr of NaN*****
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin       

In [9]:

# Features Name, Ticket, Cabin and Embarked can be dropped and they will not have significant impact on the training 
# of the K-Means model (have no impact on the survival status of the passengers).

train = train.drop(['Name','Ticket', 'Cabin','Embarked'], axis=1)
test = test.drop(['Name','Ticket', 'Cabin','Embarked'], axis=1)

labelEncoder = LabelEncoder()
labelEncoder.fit(train['Sex'])
labelEncoder.fit(test['Sex'])
train['Sex'] = labelEncoder.transform(train['Sex'])
test['Sex'] = labelEncoder.transform(test['Sex'])

print("\n")
print("*****Engineering of train info*****")
train.info()




*****Engineering of train info*****
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int32  
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
dtypes: float64(2), int32(1), int64(5)
memory usage: 52.3 KB


In [10]:

## Training of the K-Means model
## Droops the survival data
y = np.array(train['Survived'])
X = np.array(train.drop(['Survived'], axis=1).astype(float)) 

## Build the K-Means model
## Cluster the passenger records into 2: Survived or Not survived
kmeans = KMeans(n_clusters=2)
kmeans.fit(X)
KMeans(algorithm='lloyd', copy_x=True, init='k-means++', max_iter=300, n_clusters=2, n_init=10, random_state=None, tol=0.0001, verbose=0)

correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans.predict(predict_me)
    if prediction[0] == y[i]:
        correct += 1

print("Simpel model")
print(correct/len(X) )# 0.5084175084175084
print("\n")


Simpel model
0.49270482603815935




c:\Users\Wille\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [11]:

kmeans = kmeans = KMeans(n_clusters=2, max_iter=600, algorithm = 'lloyd')
kmeans.fit(X)
KMeans(algorithm='lloyd', copy_x=True, init='k-means++', max_iter=600, n_clusters=2, n_init=10, random_state=None, tol=0.0001, verbose=0)

correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans.predict(predict_me)
    if prediction[0] == y[i]:
        correct += 1

print("Simpel model (tweaked)")
print(correct/len(X))
print("\n")


c:\Users\Wille\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Simpel model (tweaked)
0.49158249158249157




In [34]:

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
kmeans.fit(X_scaled)
KMeans(algorithm='lloyd', copy_x=True, init='k-means++', max_iter=1000,
 n_clusters=2, n_init=100, random_state=None, tol=0.0001, verbose=0)
 # algorithm{“lloyd”, “elkan”, “auto”, “full”}, default=”lloyd”}
 # max_iter
correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans.predict(predict_me)
    if prediction[0] == y[i]:
        correct += 1

print("Simpel model (tweaked and range [0,1])")
print(correct/len(X)) # 0.6262626262626263


Simpel model (tweaked and range [0,1])
0.6262626262626263


c:\Users\Wille\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
